In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
import parmap
import numpy as np
import pandas as pd
import os
import parmap
import shutil
import cv2
import scipy.io as sio
from Specgram import Specgram
import csv

import glob2

from numba import jit
from sklearn.svm import SVC # "Support vector classifier"

colors = [
'black','blue','red','green', 'cyan','orange','brown','slategrey','darkviolet','darkmagenta',
'lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']


Autosaving every 180 seconds


In [2]:
def load_mask(fname):
    # load mask for dataset 
    fname_mask = fname
    for k in range(3):
        fname_mask = os.path.split(fname_mask)[0]
    fname_mask+="/genericmask.txt"
    print (fname_mask)

    mask = np.int32(np.loadtxt(fname_mask))
    raw = np.ones((128,128),'float32')
    for k in range(mask.shape[0]):
        raw[mask[k][0], mask[k][1]]=np.nan
#     plt.imshow(raw)
#     plt.show()

    return raw

In [3]:
# Initialize filenames and masks
#root_dir = '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb17_30Hz/'
root_dir = '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/'
fname = root_dir + os.path.split(root_dir[:-1])[1]+'_aligned.npy'

mask_original = load_mask(fname)


/media/cat/4TBSSD/yuki/IA2/genericmask.txt


OSError: /media/cat/4TBSSD/yuki/IA2/genericmask.txt not found.

In [5]:
plt.imshow(mask_original)
plt.show()

In [6]:
data = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/triggers/starts_grooming.npy')
print (data)


[   2.73333333    8.           16.66666667   56.26666667  125.46666667
  218.86666667  275.66666667  289.4         307.8         340.6
  416.6         435.66666667  452.2         555.06666667  782.
  833.66666667  878.33333333  910.8         983.66666667 1074.8
 1100.33333333 1127.2        1164.4       ]


In [9]:
# Fig 2A triggered motiffs;
data = np.load(fname)
labels = np.load(root_dir + '/triggers/labels.npy', allow_pickle=True)

window = np.load(root_dir + '/triggers/window.npy')

# LOAD ALL THE TRIGGERS FOR THE FEATURE BASED DATA;
starts_arrays = np.load(root_dir + '/triggers/starts_arrays.npy', allow_pickle=True)

# ADD GROOMING EVENTS
starts_grooming = np.float32(np.load(root_dir+'/triggers/starts_grooming.npy', allow_pickle=True))
starts_arrays = np.concatenate((starts_arrays,starts_grooming))
starts_arrays[7] = starts_grooming

# add the last label to the stack
labels = np.append(labels,'grooming')

# METRICS FOR COMPUTING EVENT TRIGGERED AVERAGES
bin_ave = 15
sample_rate = 30
n_cols = 12    

# save averages for each feature triggered set
stacks_ave_arrays = []
fname_dff = root_dir + 'dff_data.npy'
start = -3 # seconds start from trigger point
end = +10

if os.path.exists(fname_dff)==False:
    # select features
    features = np.arange(len(labels))
    ctr=0
    fig=plt.figure(figsize=(20,20))
    for feature in features:
        print ("feature: ", feature)
        #feature = 1
        starts_frametime = np.int32(starts_arrays[feature]*sample_rate)
        #print (labels[feature], ", starts_frametime: ", starts_frametime)


        # get neural activity stack with extra 3 seconds at the beginning of each bit
        stack = []
        dff_window = 3
        for k in range(starts_frametime.shape[0]):
            try:
                temp = data[starts_frametime[k]+int(start*sample_rate)-dff_window*sample_rate: starts_frametime[k]+int(end*sample_rate)]

                # only add events that are correct width; correct for data falling off edges;
                if temp.shape[0] == int(end-start)*sample_rate+dff_window*sample_rate:
                    stack.append(temp)
            except:
                pass
        stack = np.array(stack)

        # P
        print ("LOaded stack: ", stack.shape)

        # compute DF/F

        if True:
            stack_dff = np.zeros((stack.shape[0], int(end-start)*sample_rate, stack.shape[2], stack.shape[3]),'float32')
            print ("stack ave: ", stack_dff.shape)
            for j in range(stack.shape[0]):
                for k in range(128):
                    for p in range(128):
                        F0 = stack[j,:dff_window*sample_rate,k,p].mean(0)
                        stack_dff[j,:,k,p] = (stack[j,dff_window*sample_rate:,k,p]-F0)/F0

            #stack_ave = stack_dff.mean(0)
            stack_ave = np.median(stack_dff,axis=0)
        else:
            # compute DF/F
            stack_ave = stack.mean(0)
            for k in range(128):
                for p in range(128):
                    stack_ave[:,k,p] = stack_ave[:,k,p]-stack_ave[:stack_ave.shape[0]//2,k,p].mean(0)           

        stacks_ave_arrays.append(stack_ave)


        vmax = np.max(np.abs(stack_ave))
        for k in range(0,180,bin_ave):
            #print (len(features),n_cols,k//bin_ave+1+n_cols*ctr)
            ax=plt.subplot(len(features),n_cols,k//bin_ave+1+n_cols*ctr)
            if k==0:
                plt.ylabel(labels[feature]+": "+str(starts_frametime.shape[0]),fontsize=10)
            plt.imshow(stack_ave[k:k+bin_ave].mean(0)*mask_original, vmin=-vmax, vmax=vmax)
            plt.xticks([])
            plt.yticks([])
            #plt.title(str(round((k-stack.shape[1]/2.)/30.,2)))
        ctr+=1

    #plt.savefig('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/triggers/triggered.png',
    #           dpi=200)
    plt.show()
    
    np.save(fname_dff, stacks_ave_arrays)
else:
    stacks_ave_arrays = np.load(fname_dff)

    print ("Fisnihed loading arrays")

print (stacks_ave_arrays[0].shape)
print (len(stacks_ave_arrays))
print ("DONE")

Fisnihed loading arrays
(390, 128, 128)
8
DONE


In [9]:
# Visualize PCA space 

# compute PCA on all trials all times 

# which features to use to make PCA matrix; too many will take too long
features = [0,5]

if True:
    stack_out = []
    for feature in features:
        starts_frametime = np.int32(starts_arrays[feature]*sample_rate)

        stack = []
        for k in range(starts_frametime.shape[0]):
            try:
                temp = data[starts_frametime[k]-int(window*sample_rate): starts_frametime[k]+int(window*sample_rate)]
                if temp.shape[0] == int(window*sample_rate*2):
                    stack.append(temp)
            except:
                pass

        stack = np.array(stack)
        print (stack.shape)
        stack_3D= stack.reshape(stack.shape[0],stack.shape[1],-1)
        print ("stack 3d: ", stack_3D.shape)

        stack_in = []
        for k in range(stack_3D.shape[0]):
            stack_in.extend(stack_3D[k])
        stack_in=np.vstack(stack_in)
        print ("stack in: ", stack_in.shape)
        stack_out.append(stack_in)

    stack_out = np.vstack(stack_out)
    print ("stack_out: ", stack_out.shape)
    # run PCA on all data
    from sklearn.decomposition import PCA

    pca = PCA(n_components=100)
    pca_model = pca.fit(stack_out)

    PCS = pca_model.transform(stack_out)
    print ("PCS out: ", PCS.shape)
    print ("DONE PCA Block")

# compute PCA only on averages
else:
    
    stack_out = []
    for feature in features:
        starts_frametime = np.int32(starts_arrays[feature]*sample_rate)

        stack = []
        for k in range(starts_frametime.shape[0]):
            try:
                temp = data[starts_frametime[k]-int(window*sample_rate): starts_frametime[k]+int(window*sample_rate)]
                if temp.shape[0] == int(window*sample_rate*2):
                    stack.append(temp)
            except:
                pass

        stack = np.array(stack)
        print (stack.shape)
        stack_3D= stack.reshape(stack.shape[0],stack.shape[1],-1)
        print ("stack 3d: ", stack_3D.shape)

        stack_in=stack_3D.mean(0)
        print ("stack in: ", stack_in.shape)
        stack_out.append(stack_in)

    stack_out = np.vstack(stack_out)
    print ("stack_out: ", stack_out.shape)
    # run PCA on all data
    from sklearn.decomposition import PCA

    pca = PCA(n_components=100)
    pca_model = pca.fit(stack_out)

    PCS = pca_model.transform(stack_out)
    print ("PCS out: ", PCS.shape)
    print ("DONE PCA Block")
    
print ("DONE PROCESSING PCA ")
    

(84, 180, 128, 128)
stack 3d:  (84, 180, 16384)
stack in:  (15120, 16384)
(65, 180, 128, 128)
stack 3d:  (65, 180, 16384)
stack in:  (11700, 16384)
stack_out:  (26820, 16384)
PCS out:  (26820, 100)
DONE PCA Block
DONE PROCESSING PCA 


In [10]:
fig=plt.figure()
plt.scatter(PCS[:,0], PCS[:,1],alpha=.1)
plt.show()

In [11]:
# plot explained variance
fig=plt.figure()
expl_variance = pca_model.explained_variance_[:10]
plt.scatter(np.arange(expl_variance.shape[0]), expl_variance/expl_variance.sum(0),s=1000)
plt.ylim(0,1)
plt.ylabel("Explained Variance",fontsize=40)
plt.xlabel("Principal component",fontsize=40)
plt.tick_params(axis = 'both', which = 'major', labelsize = 40)

plt.show()

In [12]:
mask = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_aligned_maskwarp.npy')
if True:
    idx = np.where(mask!=0)
    mask[idx]=1

    idx = np.where(mask==0)
    mask[idx]=np.nan
#mask[~idx]=1
plt.imshow(mask)
plt.show()

In [36]:
print (pca_model.components_.shape)
print (pca_model.singular_values_[:10]**2)

(100, 16384)
[9.75446591e+12 8.48452550e+11 1.89249936e+11 8.85156429e+10
 7.59601754e+10 5.69121674e+10 2.88994562e+10 1.97761603e+10
 1.49791553e+10 1.03343404e+10]


In [13]:
# load the components
loadings = pca_model.components_.T #* np.sqrt(pca_model.explained_variance_)
print (loadings.shape)

# 
plt.title("First 20 Loadings",fontsize=40)
for k in range(9):
    ax = plt.subplot(3,3,k+1)
    plt.title("Comp: "+str(k))
    
    temp = loadings[:,k].reshape(128,128)
    #vmax = np.max(np.abs(temp))
    plt.imshow(temp*mask_original)
    plt.xticks([])
    plt.yticks([])
    
plt.suptitle("PC component * explained variance ")
plt.show()


(16384, 100)


In [14]:
# FIG 2C:  Dynamics PC1 vs PC2

ctr=0
clrs = ['green','blue','brown','black','red','cyan','magenta','yellow']

features = np.arange(8)
stack_out = []
#for feature in features:
fig = plt.figure(figsize=(10,10))
#for feature in features:
pcX = 0
pcY = 1
for feature in [0]:
#for feature in features:    # compute start times of selected feature in seconds
    starts_frametime = np.int32(starts_arrays[feature]*sample_rate)

    # grab raw imaging frames 
    stack = []
    for k in range(starts_frametime.shape[0]):
        try:
            temp = data[starts_frametime[k]-int(window*sample_rate): starts_frametime[k]+int(window*sample_rate)]
            if temp.shape[0] == int(window*sample_rate*2):
                stack.append(temp)
        except:
            pass

    stack = np.array(stack)
    print ("stack shape: ", stack.shape)
    
    # flatten last 2 dimensions
    stack_3D= stack.reshape(stack.shape[0],stack.shape[1],-1)
    print ("stack 3d: ", stack_3D.shape)

    # run PCA on single trials
    if False:
        stack_in = []
        for k in range(stack_3D.shape[0]):
            stack_in.extend(stack_3D[k])
        stack_in=np.vstack(stack_in)
        
        # convert data to PCs
        PCS_feature = pca_model.transform(stack_in)
        print ("PCS feature: ", PCS_feature.shape)
        pcs_3d = []
        for p in range(0,PCS_feature.shape[0], stack_3D.shape[1]):
            pcs_3d.append(PCS_feature[p:p+stack_3D.shape[1]])
        pcs_3d = np.array(pcs_3d)
        print (pcs_3d.shape)
                
        ave = []
        for k in range(pcs_3d.shape[0]):
            ave.append(pcs_3d[k,:])

        ave= np.array(ave)
        colors = plt.cm.viridis(np.linspace(0,1,ave.shape[1]))

        ave_ave = ave.mean(0)
        plt.scatter(ave_ave[0,pcX], ave_ave[0,pcY], s=500, alpha=1, c=clrs[ctr])
        if False:
            for k in range(0,ave_ave.shape[0]-1,1):
                plt.plot(ave_ave[k:k+2,pcX], ave_ave[k:k+2,pcY], linewidth=5, 
                         color=cmap(k))
        else:
            for k in range(0,ave_ave.shape[0]-1,1):
                
                if k==0:
                    plt.plot(ave_ave[k:k+2,pcX], ave_ave[k:k+2,pcY], linewidth=5, 
                        color=clrs[feature],
                        alpha=1,label=labels[feature])
                else:
                    plt.plot(ave_ave[k:k+2,pcX], ave_ave[k:k+2,pcY], linewidth=5, 
                        color=clrs[feature],
                        alpha=1-(k/ave_ave.shape[0]*0.35+.35))

        #plt.text(ave_ave[90,0]+1000, ave_ave[90,1],'t=0sec')
        plt.scatter(ave_ave[90,pcX], ave_ave[90,pcY], s=500, hatch='++', alpha=1, c=clrs[ctr])
        plt.scatter(ave_ave[179,pcX], ave_ave[179,pcY], s=500, hatch='///', alpha=1, c=clrs[ctr])

        plt.legend(fontsize=20)
    # run PCA on average only
    else:
        
        #ax=plt.subplot(2,4,feature+1)
        ax=plt.subplot(1,1,1)
        
        # RUN PCA ON THE AVERAGE OF ALL TRACES IN A STACK
        stack_in=stack_3D.mean(0)
        PCS_feature = pca_model.transform(stack_in)
        print ("PCS feature: ", PCS_feature.shape)
        pcs_3d = []
        for p in range(0,PCS_feature.shape[0], stack_3D.shape[1]):
            pcs_3d.append(PCS_feature[p:p+stack_3D.shape[1]])
        pcs_3d = np.array(pcs_3d).squeeze()
        print ("pcs 3D: ", pcs_3d.shape)
        
        
        # *****************************************
        # PLOT STEP
        #colors = plt.cm.viridis(np.linspace(0,1,pcs_3d.shape[0]))
        cmap = cm.get_cmap('viridis',pcs_3d.shape[0])

        plt.scatter(pcs_3d[0,pcX], pcs_3d[0,pcY], s=250, alpha=1, c=clrs[ctr])
        # plot bits at a time to use alpha
        if False: 
            for k in range(0,pcs_3d.shape[0]-1,1):
                #plt.plot(pcs_3d[k:k+2,0], pcs_3d[k:k+2,1], linewidth=5, color=cmap(k))
                plt.plot(pcs_3d[k:k+2,pcX], pcs_3d[k:k+2,pcY], linewidth=5, color=clrs[ctr],alpha=(180-k)/200)
        else:
            plt.plot(pcs_3d[:,pcX], pcs_3d[:,pcY], linewidth=6, color=clrs[ctr],alpha=.7)
            
        # plot the locations of other features in the timeseries
        for j in features:
            
            # skip self triggered location
            if j==feature:
                continue
            starts_primary_features = starts_arrays[feature]
            
            # find other features starts in the same window as the main features selected:
            starts_secondary_features = starts_arrays[j]
            
            # loop over each starts_frametime
            # loop over each starts_frametime
            print ("window: ", window)
            print (" starts_secondary_features: ", starts_secondary_features[:10])
            print (" starts_primary_features: ", starts_primary_features[:10])
            
            for p in range(starts_primary_features.shape[0]):
                idx = np.where(np.logical_and(starts_secondary_features>=starts_primary_features[p]-window,
                                              starts_secondary_features<=starts_primary_features[p]+window))
                
                secondary_starts_seconds = starts_secondary_features[idx]-starts_primary_features[p]+window
                #print ("feature: ", labels[j], " secondary starts seconds; ", secondary_starts_seconds)

                secondary_starts_frames = np.int32(secondary_starts_seconds*sample_rate)
                idx = np.where(np.logical_and(secondary_starts_frames>=0, secondary_starts_frames<180))[0]
                secondary_starts_frames=secondary_starts_frames[idx]

                #print ("feature: ", labels[j], " secondary starts frames; ", secondary_starts_frames)
                for k in secondary_starts_frames:
                    plt.scatter(pcs_3d[k,pcX]+np.random.rand()*1000-500, 
                                pcs_3d[k,pcY]+np.random.rand()*1000-500, 
                                s=10, 
                                color=clrs[j],
                               alpha=.8)
        
        # plot star
        plt.scatter(pcs_3d[0,pcX], pcs_3d[0,pcY], s=250, c=clrs[ctr], edgecolors='black',alpha=.5)
        plt.scatter(pcs_3d[pcs_3d.shape[0]//2,pcX], pcs_3d[pcs_3d.shape[0]//2,pcY], s=250, c=clrs[ctr], edgecolors='black',alpha=.5)
        plt.scatter(pcs_3d[-1,pcX], pcs_3d[-1,pcY], s=250, c=clrs[ctr], edgecolors='black', alpha=.5)
        plt.text(pcs_3d[0,pcX]+1000, pcs_3d[0,pcY],'t=-3sec',fontsize=20)
        plt.text(pcs_3d[pcs_3d.shape[0]//2,pcX]+1000, pcs_3d[pcs_3d.shape[0]//2,pcY],'t=0sec',fontsize=20)
        plt.text(pcs_3d[-1,pcX]+1000, pcs_3d[-1,pcY],'t=+3sec',fontsize=20)

        
        # Format plot
        plt.xlabel("PC"+str(pcX+1),fontsize=20)
        plt.ylabel("PC"+str(pcY+1),fontsize=20)
        #plt.xlim(-10000,12000)
        #plt.ylim(-10000,12000)

        #plt.plot([0,0],[-10000,12000],'r--', c='black')
        #plt.plot([-10000,12000],[0,0],'r--',c='black')
   
    plt.title(labels[feature])
    
    ctr+=1
if False:
    plt.savefig('/home/cat/tcs.png',dpi=100)
    plt.close()
else:
    plt.show()

stack shape:  (84, 180, 128, 128)
stack 3d:  (84, 180, 16384)
PCS feature:  (180, 100)
pcs 3D:  (180, 100)
window:  3.0
 starts_secondary_features:  [1.33333333e-01 1.90666667e+01 3.03333333e+01 3.99333333e+01
 7.80000000e+01 9.62666667e+01 1.05133333e+02 1.13600000e+02
 1.23133333e+02 1.38933333e+02]
 starts_primary_features:  [1.33333333e-01 1.63333333e+01 4.00000000e+01 5.08000000e+01
 6.90000000e+01 9.60666667e+01 1.05133333e+02 1.23066667e+02
 1.38466667e+02 1.47066667e+02]
window:  3.0
 starts_secondary_features:  [  0.73333333  16.6         37.66666667  65.73333333  69.
  95.8        123.         138.46666667 173.13333333 187.2       ]
 starts_primary_features:  [1.33333333e-01 1.63333333e+01 4.00000000e+01 5.08000000e+01
 6.90000000e+01 9.60666667e+01 1.05133333e+02 1.23066667e+02
 1.38466667e+02 1.47066667e+02]
window:  3.0
 starts_secondary_features:  [ 0.13333333 19.6        30.6        40.         52.26666667 63.2
 68.73333333 78.06666667 84.66666667 96.2       ]
 starts_pr



########### VISUALIZE REGISTERED REGIONS ###########



In [2]:
mask = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_aligned_maskwarp.npy')
if True:
    idx = np.where(mask!=0)
    mask[idx]=1

    idx = np.where(mask==0)
    mask[idx]=np.nan
#mask[~idx]=1
plt.imshow(mask)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_aligned_maskwarp.npy'

In [3]:
# load REGISTERED MAP IN 679 x 586 resolution

dorsalMaps_unique = np.load('/home/cat/dorsalMaps_name.npy')
print (dorsalMaps_unique.shape)
print (np.unique(dorsalMaps_unique))
plt.imshow(dorsalMaps_unique)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/cat/dorsalMaps_name.npy'

In [12]:
# load REGISTERED MAP IN 128 x 128 resolution
# NOTE: This map contains average errors that need to be removed:
# so replace the values which are not in the dorsalMap list above with most frequent nearby values;

# fix mean maskwarp values
maskwarp = np.load('/home/cat/maskwarp.npy')
ax = plt.subplot(1,2,1)
plt.imshow(maskwarp)
print (np.unique(maskwarp))
maskwarp_unique_ids = np.unique(maskwarp)
for id_ in maskwarp_unique_ids:
    #print (id_)
    if id_ in dorsalMaps_unique:
        pass
    else:
        idx = np.where(maskwarp==id_)
        
        for k in range(len(idx[0])):
            maskwarp[idx[0][k],idx[1][k]] = np.nan

        for k in range(len(idx[0])):
            temp = maskwarp[max(0,idx[0][k]-1): idx[0][k]+2, max(0,idx[1][k]-1): idx[1][k]+2].ravel()
            uniques = np.unique(temp, return_counts=True)
            idx4 = np.argmax(uniques[1])
            val = uniques[0][idx4]  # Replace values with most frequent nearby vals;
            maskwarp[idx[0][k],idx[1][k]] = val

ax = plt.subplot(1,2,2)
plt.imshow(maskwarp)
print (np.unique(maskwarp))
plt.show()

[  0.   1.   2.   3.   4.   5.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.
  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  55.  56.  57.
  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.
  72.  73.  74.  75.  76.  78.  79.  80.  81.  83.  84.  85.  86.  88.
  90.  91.  92.  93.  94.  95.  96.  97.  99. 100. 101. 102. 103. 104.
 106. 107. 108. 113. 114. 115. 116. 117. 119. 120. 121. 123. 124. 125.
 127. 128. 129. 130. 132. 134. 135. 136. 137. 138. 139. 140. 141. 142.
 143. 144. 146. 147. 148. 149. 150. 151. 152. 153. 154. 155. 156. 157.
 158. 159. 160. 161. 162. 163. 164. 165. 166. 167. 168. 170. 171. 172.
 173. 174. 175. 176. 177. 178. 179. 180. 182. 183. 184. 185. 186. 187.
 188. 189. 190. 191. 194. 195. 196. 197. 198. 199. 200. 201. 203. 204.
 206. 208. 210. 214. 215. 216. 217. 218. 220. 221. 222. 223. 225. 228.
 229. 

In [4]:

# load names/labels for data
ids = np.load('/home/cat/dorsalMaps_id.npy').squeeze()
print ("All Allen IDS: ", ids.shape[0])
import csv
names = []
with open('/home/cat/dorsalMaps_name.txt', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        names.append(row)
        
print ("All Allen area names: ", len(names[1:]))



FileNotFoundError: [Errno 2] No such file or directory: '/home/cat/dorsalMaps_id.npy'

In [16]:
# visualize annotated areas:
registered_ids = np.int32(np.unique(maskwarp))
print ("Registered area ids: ", registered_ids.shape)
print ("registered ids: ", registered_ids[:10])

ctr=1
areas = [15, 21, 43, 57,150, 249, 255,261, 268, 275]
area_masks = []
names_array = []
fig=plt.figure()
#for k in range(25,registered_ids.shape[0],25):
for k in range(0,registered_ids.shape[0],1):
    ax=plt.subplot(4,10,ctr)
    temp = maskwarp.copy()

    print ("searching for ", registered_ids[k])
    idx = np.where(temp!=registered_ids[k])
    temp*=1
    temp[idx]=0
    temp*=mask
    if registered_ids[k] in areas:
        area_masks.append(idx)
        names_array.append(names[registered_ids[k]+1])
    plt.imshow(temp)
    
    ctr+=1
    # label stuff
    
    titles = ''
    for p in range(len(names[registered_ids[k]+1])):
        titles = titles + names[registered_ids[k]+1][p]+'\n'
    
    plt.ylabel(str(k)+" "+str(registered_ids[k]))
    plt.xticks([])
    plt.yticks([])
    plt.title(titles,fontsize=8,y=0.85)
plt.show()

Registered area ids:  (39,)
registered ids:  [ 0  8 15 21 29 36 43 50 57 64]
searching for  0
searching for  8
searching for  15
searching for  21
searching for  29
searching for  36
searching for  43
searching for  50
searching for  57
searching for  64
searching for  71
searching for  78
searching for  92
searching for  100
searching for  107
searching for  114
searching for  121
searching for  129
searching for  136
searching for  143
searching for  150
searching for  157
searching for  164
searching for  171
searching for  178
searching for  186
searching for  198
searching for  217
searching for  249
searching for  255
searching for  261
searching for  268
searching for  275
searching for  282
searching for  295
searching for  300
searching for  301
searching for  651
searching for  653


In [18]:
print (len(names_array))
print (names_array)


10
[['"Primary', 'motor', 'area,', 'Layer', '2/3"'], ['"Secondary', 'motor', 'area,', 'layer', '2/3"'], ['"Primary', 'somatosensory', 'area,', 'lower', 'limb,', 'layer', '2/3"'], ['"Primary', 'somatosensory', 'area,', 'upper', 'limb,', 'layer', '2/3"'], ['"Primary', 'visual', 'area,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'lateral', 'agranular', 'part,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'dorsal', 'part,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'ventral', 'part,', 'layer', '2/3"'], ['"Anterior', 'area,', 'layer', '2/3"'], ['"Rostrolateral', 'area,', 'layer', '2/3"']]


In [19]:
# grab masks for RS dorsal and RS ventral 
img = np.zeros((128,128),'float32') #*np.nan
print (len(area_masks))
masks_2D = []
for k in range(len(area_masks)):
    print (k)
    ax=plt.subplot(4,3,k+1)
    temp =img.copy()
    temp[area_masks[k]]=np.nan
    masks_2D.append(temp)
    plt.imshow(temp)
    plt.title(names_array[k])
plt.show()

np.save('/home/cat/areas.npy', masks_2D)
np.save('/home/cat/area_names.npy',names_array)

10
0
1
2
3
4
5
6
7
8
9


/home/cat/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


/home/cat/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [20]:
# USE stacks_ave_arrays ([# features, # times steps, width, height]) to visualize specific areas:
# to plot ROI vs time

temp_temp = np.array(stacks_ave_arrays)
print (temp_temp.shape)

selected_feature = 0
t=np.arange(temp_temp.shape[1])/30. - 3
ax = plt.subplot(2,1,1)
for k in range(1,registered_ids.shape[0],1):

    temp = maskwarp.copy()
    
    print ("searching for area ", registered_ids[k])
    # find indexes for particaulr ROI
    idx = np.where(temp==registered_ids[k])
    #time_series = temp_temp[selected_feature,:, :].mean(1).mean(1)#, idx[0], idx[1]].mean(0)
    time_series = temp_temp[selected_feature, :, idx[0], idx[1]].mean(0)
    print (time_series.shape)
    
    #for p in range(len(names[registered_ids[k]])):
    #    if "Retrosplenial" in names[registered_ids[k]][p]:
    plt.plot(t, time_series, label=names[registered_ids[k]])

#plt.legend(fontsize=10)
ax = plt.subplot(2,1,2)
time_series = temp_temp[selected_feature]
print (time_series.shape)
temp_show = np.zeros((128,128),'float32')
for k in range(time_series.shape[0]):
    #print (time_series[k].shape, mask.shape, (time_series[k]*mask).shape)
    temp_show= np.hstack((temp_show, time_series[k]*mask))

#temp_show = #np.hstack(temp_show)
print ("final: ", temp_show.shape)
plt.imshow(temp_show,aspect='auto')
    
    
plt.suptitle("Trigger: "+str(labels[selected_feature]))
plt.show()

(8, 390, 128, 128)
searching for area  8
(390,)
searching for area  15
(390,)
searching for area  21
(390,)
searching for area  29
(390,)
searching for area  36
(390,)
searching for area  43
(390,)
searching for area  50
(390,)
searching for area  57
(390,)
searching for area  64
(390,)
searching for area  71
(390,)
searching for area  78
(390,)
searching for area  92
(390,)
searching for area  100
(390,)
searching for area  107
(390,)
searching for area  114
(390,)
searching for area  121
(390,)
searching for area  129
(390,)
searching for area  136
(390,)
searching for area  143
(390,)
searching for area  150
(390,)
searching for area  157
(390,)
searching for area  164
(390,)
searching for area  171
(390,)
searching for area  178
(390,)
searching for area  186
(390,)
searching for area  198
(390,)
searching for area  217
(390,)
searching for area  249
(390,)
searching for area  255
(390,)
searching for area  261
(390,)
searching for area  268
(390,)
searching for area  275
(390,)
se

[  0   8  15  16  20  21  29  36  43  50  57  64  71  78 100 114 129 136
 143 150 157 164 171 178 186 198 249 255 261 268 275 282 300 301 653]


In [32]:
# load the ROI areas used for SVM analysis
ROI_area_ids = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_Mar4_30Hz/IA1am_Mar4_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec_area_ids.npy')
print (ROI_area_ids)

# load the names of all 840 allen ROIs
fname = '/media/cat/4TBSSD/yuki/meta_allenmap/dorsalMaps_name.txt'

# get the names of the ROI_area_ids
import csv
names = []
with open(fname) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        names.append(row)

# print the matched ids
names=np.array(names)
for id_ in ROI_area_ids:
    print (id_, names[id_])

840
0 ['name']
8 ['Frontal pole, layer 1']
15 ['Primary motor area, Layer 1']
16 ['Primary motor area, Layer 2/3']
20 ['Secondary motor area']
21 ['Secondary motor area, layer 1']
29 ['Primary somatosensory area, nose, layer 1']
36 ['Primary somatosensory area, barrel field, layer 1']
43 ['Primary somatosensory area, lower limb, layer 1']
50 ['Primary somatosensory area, mouth, layer 1']
57 ['Primary somatosensory area, upper limb, layer 1']
64 ['Primary somatosensory area, trunk, layer 1']
71 ['Primary somatosensory area, unassigned, layer 1']
78 ['Supplemental somatosensory area, layer 1']
100 ['Dorsal auditory area, layer 1']
114 ['Posterior auditory area, layer 1']
129 ['Anterolateral visual area, layer 1']
136 ['Anteromedial visual area, layer 1']
143 ['Lateral visual area, layer 1']
150 ['Primary visual area, layer 1']
157 ['Posterolateral visual area, layer 1']
164 ['posteromedial visual area, layer 1']
171 ['Laterointermediate area, layer 1']
178 ['Postrhinal area, layer 1']
18

In [63]:
# make visualizatoin of all the ROI areas

# set backend to not plot and save directly; allows for higher res figs
matplotlib.use('Agg')

# load the maskwarp we used
mask = np.load('/media/cat/4TBSSD/yuki/meta_allenmap/maskwarp.npy')

# plot data
ctr=1
fig=plt.figure(figsize=(30,30))
for k in ROI_area_ids:
    ax=plt.subplot(5,7,ctr)
    idx = np.where(mask==k)
    mask1 = mask.copy()
    mask1[idx]=np.nan
    plt.imshow(mask1)
    ctr+=1
    plt.xticks([])
    plt.yticks([])
    plt.title(str(k)+" "+str(names[k]), fontsize=9)
plt.savefig('/home/cat/maps.png')
plt.close()